# Честная регрессия


In [41]:
import numpy as np
class TrueLinReg():
    def __init__(self):
        self.w_ = None  #cтолбец 
        self.coef_ = None #строка
        
    def fit(self, X: np.array, y: np.array) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1) #добавляем вес без признака        
        n, m = X.shape[0], X.shape[1]
        y = np.array(y)
        y = y.reshape(n, 1)
        
        if n == m:
            self.w_ = np.linalg.inv(X).dot(y)
        elif n > m:
            first = X.T.dot(X)
            second = np.linalg.inv(first)
            third = second.dot(X.T)
            self.w_ = third.dot(y)
        elif n < m:
            first = X.dot(X.T)
            second = np.linalg.inv(first)
            third = X.T.dot(second)
            self.w_ = third.dot(y)
        
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        return (X @ self.w_).reshape(-1)

model = TrueLinReg()

X_train = np.array([[1], [2], [3]])
y_train = np.array([1, -2, 1])
X_test = np.array([[0], [4]])

model.fit(X_train, y_train)
print(model.predict(X_test))
print(model.w_)

print(model.predict(X_test) == np.array([0., 0.]))
print(model.w_ == np.array([[0.], [0.]]))

X_train = np.array([[1]])
y_train = np.array([2])
X_test = np.array([[0.], [2.], [3.]])

model.fit(X_train, y_train)
print(model.predict(X_test))
print(model.w_)

print(model.predict(X_test) == np.array([1., 3., 4.]))
print(model.w_ == np.array([[1.], [1.]]))

[ 0.0000000e+00 -4.4408921e-16]
[[-1.11022302e-16]
 [ 0.00000000e+00]]
[ True False]
[[False]
 [ True]]
[1. 3. 4.]
[[1.]
 [1.]]
[ True  True  True]
[[ True]
 [ True]]


# Честная регуляризация

Добавьте регуляризацию с коэффициентом λ и решите задачу регрессии (для любого соотношения m и n используйте формулу)

$$\omega = (X^TX+\frac{\lambda}{2}I)^{-1}X^TY$$

где I - единичная матрица.

Не забудьте сами добавить справа единичный столбец к матрице X аналогично предыдущей задачи.

In [115]:
class TrueReg():
    def __init__(self, lamb):
        self.lamb = lamb
        self.w_ = None #столбец
        self.coef_ = None #строка
        
    def fit(self, X, y):
        X = self._add_ones_at_right(X)
        n, m = X.shape[0], X.shape[1]
        y = y.reshape(n, 1)
        ones = np.eye(m)
        
        self.w_ = np.linalg.inv(X.T.dot(X) + self.lamb * ones).dot(X.T).dot(y)
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X):
        X = self._add_ones_at_right(X)
        return (X @ self.w_).reshape(-1)
    
    def _add_ones_at_right(self, X):
        return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)

    
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])
lamb = 1

X_test = np.array([[0], [4]])

model = TrueReg(lamb)

model.fit(X_train, y_train)
print(model.predict(X_test)) # np.array([0.33, 3])
print(model.w_) # np.array([[0.67], [0.33]])
print(model.coef_) # np.array([0.67, 0.33])

[0.33333333 3.        ]
[[0.66666667]
 [0.33333333]]
[0.66666667 0.33333333]


In [54]:
np.array([[1, 2], [3, 4]]) + (1/2 * np.eye(2))

array([[1.5, 2. ],
       [3. , 4.5]])

# Градиент

На вход подаются обучающая выборка (X,y) и как-то определенные веса w. Верните градиент функции MSE для изменения весов в алгоритме градиентного спуска.

$$\nabla L=\frac{2}{n}X^T(Xw-y)$$

Здесь не нужно добавлять единицы сбоку к матрице, считаем, что они уже есть.

In [88]:
def gradient_step(w: np.array, X: np.array, y: np.array) -> np.array:
    n = X.shape[0]
    w = w.reshape(w.shape[0], 1)
    return (2 / n) * X.T.dot(X.dot(w) - y)

w = np.array([0.2, 0.2])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[2],[3]]) #столбец!
print(gradient_step(w, X, y))

[[-5.6]
 [-5.6]]


# Стохастический градиент

Реализуйте стохаистический градиентный спуск: пересчитайте $$\nabla_{w}L$$ только для одного случайно выбранного элемента, а остальные элементы вектора пусть будут равны нулю.

$$\nabla_{w_j}L=\frac{2}{n} \sum_{i=1}^{n}x_{i, j}(< x_i, w > -y_i)$$

Здесь не нужно добавлять единицы сбоку к матрице X, считаем, что они уже есть.

In [14]:
import numpy as np
import random

def stochastic_step(w: np.array, X: np.array, y: np.array) -> np.array:
    n = X.shape[0]
    r = random.randint(0, n-1)
    rand_X = X[r]
    return 2 * rand_X * (rand_X.dot(w) - y[r])
    

w = np.array([[1], [1], [1]])
X = np.array([[1, 1, 1], [0, 0, 0]])
y = np.array([[1], [0]])

print(stochastic_step(w, X, y))

[4 4 4]


# Градиентный спуск

Теперь реализуем Линейную регрессию на градиентном спуске. Реализуйте пересчёт весов в цикле для алгоритма градиентного спуска. Начальные веса инициализированы нулями.
$$w_{new}=w - \eta \nabla L$$
Вы можете использовать из предыдущей задачи как простой градиентный спуск, так и стохастический:

Для простого лучше использовать параметры max_iter=1000, eta=0.1
Для стохастического: max_iter=100000, eta=0.05
В случае стохастического градиентного спуска есть веротяность не сойтись, в этом случае задача будет не зачтена. Попробуйте перезаслать несколько раз (если 5 раз не хватило, значит ошибка где-то в коде).

Подсказка: не забудьте остановиться если вы достигли минимума (разница между старыми и новыми весами - крайне мала).

In [107]:
class GDLinReg():
    def __init__(self, max_iter=1000, eta=0.1):
        self.max_iter = max_iter
        self.eta = eta
        self.coef_ = None #строка
        self.w_ = None #столбец
    
    def _gradient_descending(self, w, X, y):
        n = X.shape[0]
        for i in range(self.max_iter):
            self.w_ -= self.eta * (2 / n) * X.T.dot(X.dot(self.w_) - y)
    
    def fit(self, X, y):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        n, m = X.shape
        self.w_ = np.zeros((m, 1)) #столбец
        y = y.reshape(y.shape[0], 1)
        
        self._gradient_descending(w, X, y)
        
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)   
        return (X @ self.w_).reshape(-1) #возвращаем всегда строку
    
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])

model = GDLinReg(max_iter=1000, eta=0.1).fit(X_train, y_train)
y_pred = model.predict(np.array([[3],[4]]))
print(y_pred) #np.array([3., 4.])
print(model.coef_) # np.array([1., 0.])

[2.99999984 3.99999973]
[9.99999886e-01 1.85168395e-07]


# Регуляризация

Реализуйте L2-регуляризацию (так же известную как Ridge).

$$L = \frac{1}{n} \|Xw-y\|^2_2+\frac{\lambda}{2} \|w\|_2^2$$
​	
 
Найдите новый $$\nabla_{w}L$$ и верните его.

Обратите внимание, что свободный коэффициент весов (самый последний) не нужно регуляризовывать.

In [119]:
def gradient_step_l2(w: np.array, X: np.array, y: np.array, lamb: float) -> np.array:
    n = X.shape[0]
    w = w.reshape(w.shape[0], 1)
    w_without_last = np.copy(w)
    w_without_last[w.shape[0] - 1, 0] = 0
    return (2 / n) * X.T.dot(X.dot(w) - y) + lamb * w_without_last
    
w = np.array([[1], [1]])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[0],[0]])

print(gradient_step_l2(w, X, y, 1)) # np.array([[19.], [18.]])

[[19.]
 [18.]]


# Логистический градиент

Теперь переходим к задаче классификации и Логистической регрессии. Мы уже знаем, что в нашем случае нужно минимизировать метрику LogLoss:

$$lnL=\sum_{i=1}^{n}y_iln(\sigma(x_i^Tw)) + (1-y_i)ln(1-\sigma(x_i^Tw))$$, где
$$\sigma(z) = \frac{1}{1 + e^{-z}} y \in \{0,1\}$$

Ваша задача взять производную этой функции и вернуть вектор градиентов $\nabla_wlnL$

Формулу необходимо вывести в векторном виде, чтобы считалось быстрее.

Обратите внимание:

- w, y - столбцы, а не строки. Ответ тоже столбец.
- В функции используется натуральный логарифм.
- Нужно посчитать полный градиент, а не стохастический.

In [135]:
import numpy as np
import math

def ef(v):
    return math.exp(v)

def log_gradient_step(w: np.array, X: np.array, y: np.array)-> np.array:
    w = np.array(w)
    w = w.reshape(w.shape[0], 1)
    efv = np.vectorize(ef)
    e = efv(X.dot(w))
    return X.T.dot((1 - (e / (1 + e)) - y))
    
    return (y - (1-y) * e) / (1 + e)
    
w = np.array([0, 0])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[0],[0]])
log_gradient_step(w, X, y)

ValueError: shapes (3,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)